In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and prepare the dataset
file_path = 'goal_ticks.csv'
column_names = [
    "ball_X", "ball_Z", "ball_aim_X", "ball_aim_Z", "dist_to_ball", "kart_X", "kart_Z", 
    "vel_X", "vel_Z", "speed", "steer", "accel", "brake", "skid", "goal"
]

try:
    df = pd.read_csv(file_path, header=None, names=column_names)
except Exception as e:
    print(f"Error loading the data file: {e}")
    exit(1)

# Extract features and labels (assuming goal column is not needed anymore)
X = df[[
    "ball_X", "ball_Z", "ball_aim_X", "ball_aim_Z", "dist_to_ball", "kart_X", "kart_Z",
    "vel_X", "vel_Z", "speed", "steer", "accel", "brake", "skid"
]].values
y = df[['steer', 'accel']].values  # Adjust this line based on your actual data structure for outputs

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)  # Change to float32 if the outputs are continuous

# Define the neural network architecture
class SoccerAI(nn.Module):
    def __init__(self, num_inputs):
        super(SoccerAI, self).__init__()
        self.fc1 = nn.Linear(num_inputs, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)  # Output for steering and acceleration

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)  # No activation here to allow continuous output
        return x

# Initialize the model
model = SoccerAI(X_tensor.shape[1])

# Optimization and loss function setup for regression
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(train_loader, model, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Validation function
def validate_model(val_loader, model):
    model.eval()
    total_loss = 0
    count = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            count += 1
    avg_loss = total_loss / count
    print(f'Validation Loss: {avg_loss:.4f}')

# Split the data and create data loaders
X_train, X_val, y_train, y_val = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Execute training and validation
train_model(train_loader, model, criterion, optimizer, 20)
validate_model(val_loader, model)

# Save the trained model for deployment
model.eval()
example_input = torch.rand(1, X_train.shape[1])  # Generate a random example input
traced_script_module = torch.jit.trace(model, example_input)
traced_script_module.save("soccer_ai_model.pt")


Epoch 1/20, Loss: 0.0001
Epoch 2/20, Loss: 0.0000
Epoch 3/20, Loss: 0.0000
Epoch 4/20, Loss: 0.0000
Epoch 5/20, Loss: 0.0000
Epoch 6/20, Loss: 0.0000
Epoch 7/20, Loss: 0.0000
Epoch 8/20, Loss: 0.0000
Epoch 9/20, Loss: 0.0000
Epoch 10/20, Loss: 0.0000
Epoch 11/20, Loss: 0.0000
Epoch 12/20, Loss: 0.0000
Epoch 13/20, Loss: 0.0000
Epoch 14/20, Loss: 0.0000
Epoch 15/20, Loss: 0.0000
Epoch 16/20, Loss: 0.0000
Epoch 17/20, Loss: 0.0000
Epoch 18/20, Loss: 0.0000
Epoch 19/20, Loss: 0.0000
Epoch 20/20, Loss: 0.0000
Validation Loss: 0.0000
